### ***PROJECT***: (고양시) 공공자전거 스테이션 최적 위치 선정
  - [고양시_compass](https://https://compas.lh.or.kr/subj/competition/info?subjNo=SBJ_2007_001)
  - [Reference](https://https://www.kaggle.com/kwonyoung234/for-beginner)

In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns #시각화를 위한 라이브러리
import matplotlib.pyplot as plt
import calendar 
from datetime import datetime

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# Mount Google Drive #절대 바꾸지 말 것
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive #
print(ROOT)                 # print content of ROOT (Optional)
drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%cd drive/'My Drive'/'고양시'

/content/drive/My Drive/고양시


In [18]:
bicycle_now = pd.read_csv('01.운영이력.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
bicycle_now.head()

,LEAS_NO,LEAS_STAT,LEAS_DATE,LEAS_STATION,LEAS_DEF_NO,RTN_DATE,RTN_STATION,RTN_DEF_NO,TRNV_QTY,MEMB_DIV,MEMB_NO,TEMP_MEMB_NO,BIKE_TAG,RTN_PROCESS
0,15945541,2,2017-01-01 00:00:41,213,18,2017-01-01 00:13:52,260,17,0.0,6,164203,0.0,1A844000000BB7,NaN
1,15945542,2,2017-01-01 00:01:03,231,4,2017-01-01 00:50:24,231,17,31039.0,1,187551,0.0,1A844000000494,NaN
2,15945543,2,2017-01-01 00:01:50,119,14,2017-01-01 01:01:50,0,0,NaN,12,168994,0.0,1A844000000533,1.0
3,15945544,2,2017-01-01 00:02:09,121,17,2017-01-01 00:15:58,133,14,15490.0,12,183971,0.0,1A844000000731,NaN
4,15945545,2,2017-01-01 00:03:32,320,29,2017-01-01 00:18:44,259,27,0.0,12,167475,0.0,1A84400000F343,NaN


## 운영이력 데이터 컬럼
- LEAS_NO    대여 번호
- LEAS_DATE    대여 시간 !
- LEAS_STATION    대여 스테이션 번호 !
- LEAS_DEF_NO    대여 거치대 번호
- RTN_DATE    반납 시간 !
- RTN_STATION    반납 스테이션 번호 !
- RTN_DEF_NO    반납 거치대 번호
- RTN_PROCESS    관제반납구분값 (01:관제반납/02:관리자반납/03:관리자반출/04:타 사용자 반납/05:다른 사용자 반출)
- TRNV_QTY    추정 이동 거리(m) ! -> 평균 이동거리
- MEMB_DIV    회원 구분(비회원은 99이며 나머지는 정회원)
- MEMB_NO    회원 번호
- TEMP_MEMB_NO    비회원 번호
- BIKE_TAG    자전거 번호

In [20]:
#컬럼['LEAS_DDATE']와 컬럼['RTN_DATE']에서 '년월일'과 '시분초' 쪼개기
# 단, 데이터타입이 'object'여야함
bicycle_now['tempDATE']= bicycle_now.LEAS_DATE.apply(lambda x:x.split())

In [36]:
bicycle_now['tempDATE_2']= bicycle_now.RTN_DATE.apply(lambda x:x.split())

In [21]:
bicycle_now.head()

,LEAS_NO,LEAS_STAT,LEAS_DATE,LEAS_STATION,LEAS_DEF_NO,RTN_DATE,RTN_STATION,RTN_DEF_NO,TRNV_QTY,MEMB_DIV,MEMB_NO,TEMP_MEMB_NO,BIKE_TAG,RTN_PROCESS,tempDATE
0,15945541,2,2017-01-01 00:00:41,213,18,2017-01-01 00:13:52,260,17,0.0,6,164203,0.0,1A844000000BB7,NaN,"[2017-01-01, 00:00:41]"
1,15945542,2,2017-01-01 00:01:03,231,4,2017-01-01 00:50:24,231,17,31039.0,1,187551,0.0,1A844000000494,NaN,"[2017-01-01, 00:01:03]"
2,15945543,2,2017-01-01 00:01:50,119,14,2017-01-01 01:01:50,0,0,NaN,12,168994,0.0,1A844000000533,1.0,"[2017-01-01, 00:01:50]"
3,15945544,2,2017-01-01 00:02:09,121,17,2017-01-01 00:15:58,133,14,15490.0,12,183971,0.0,1A844000000731,NaN,"[2017-01-01, 00:02:09]"
4,15945545,2,2017-01-01 00:03:32,320,29,2017-01-01 00:18:44,259,27,0.0,12,167475,0.0,1A84400000F343,NaN,"[2017-01-01, 00:03:32]"


In [23]:
# tempDATE에서 년/월/일 쪼개기
bicycle_now['LEAS_year']=bicycle_now.tempDATE.apply(lambda x:x[0].split('-')[0])
bicycle_now['LEAS_month']=bicycle_now.tempDATE.apply(lambda x:x[0].split('-')[1])
bicycle_now['LEAS_day']=bicycle_now.tempDATE.apply(lambda x:x[0].split('-')[2])

In [25]:
bicycle_now.head()

,LEAS_NO,LEAS_STAT,LEAS_DATE,LEAS_STATION,LEAS_DEF_NO,RTN_DATE,RTN_STATION,RTN_DEF_NO,TRNV_QTY,MEMB_DIV,MEMB_NO,TEMP_MEMB_NO,BIKE_TAG,RTN_PROCESS,tempDATE,LEAS_year,LEAS_month,LEAS_day
0,15945541,2,2017-01-01 00:00:41,213,18,2017-01-01 00:13:52,260,17,0.0,6,164203,0.0,1A844000000BB7,NaN,"[2017-01-01, 00:00:41]",2017,01,01
1,15945542,2,2017-01-01 00:01:03,231,4,2017-01-01 00:50:24,231,17,31039.0,1,187551,0.0,1A844000000494,NaN,"[2017-01-01, 00:01:03]",2017,01,01
2,15945543,2,2017-01-01 00:01:50,119,14,2017-01-01 01:01:50,0,0,NaN,12,168994,0.0,1A844000000533,1.0,"[2017-01-01, 00:01:50]",2017,01,01
3,15945544,2,2017-01-01 00:02:09,121,17,2017-01-01 00:15:58,133,14,15490.0,12,183971,0.0,1A844000000731,NaN,"[2017-01-01, 00:02:09]",2017,01,01
4,15945545,2,2017-01-01 00:03:32,320,29,2017-01-01 00:18:44,259,27,0.0,12,167475,0.0,1A84400000F343,NaN,"[2017-01-01, 00:03:32]",2017,01,01


In [27]:
#tempDATE에서 시/분/초 쪼개기
bicycle_now['LEAStime']=bicycle_now.tempDATE.apply(lambda x:x[1].split())

In [28]:
bicycle_now.head()

,LEAS_NO,LEAS_STAT,LEAS_DATE,LEAS_STATION,LEAS_DEF_NO,RTN_DATE,RTN_STATION,RTN_DEF_NO,TRNV_QTY,MEMB_DIV,MEMB_NO,TEMP_MEMB_NO,BIKE_TAG,RTN_PROCESS,tempDATE,LEAS_year,LEAS_month,LEAS_day,LEAStime
0,15945541,2,2017-01-01 00:00:41,213,18,2017-01-01 00:13:52,260,17,0.0,6,164203,0.0,1A844000000BB7,NaN,"[2017-01-01, 00:00:41]",2017,01,01,[00:00:41]
1,15945542,2,2017-01-01 00:01:03,231,4,2017-01-01 00:50:24,231,17,31039.0,1,187551,0.0,1A844000000494,NaN,"[2017-01-01, 00:01:03]",2017,01,01,[00:01:03]
2,15945543,2,2017-01-01 00:01:50,119,14,2017-01-01 01:01:50,0,0,NaN,12,168994,0.0,1A844000000533,1.0,"[2017-01-01, 00:01:50]",2017,01,01,[00:01:50]
3,15945544,2,2017-01-01 00:02:09,121,17,2017-01-01 00:15:58,133,14,15490.0,12,183971,0.0,1A844000000731,NaN,"[2017-01-01, 00:02:09]",2017,01,01,[00:02:09]
4,15945545,2,2017-01-01 00:03:32,320,29,2017-01-01 00:18:44,259,27,0.0,12,167475,0.0,1A84400000F343,NaN,"[2017-01-01, 00:03:32]",2017,01,01,[00:03:32]


In [29]:
# tempDATE에서 시/분/초 쪼개기
# 시/분/초 단위 전처리는 필요 없을 듯함
bicycle_now['LEAS_hour']=bicycle_now.tempDATE.apply(lambda x:x[1].split(':')[0])
bicycle_now['LEAS_min']=bicycle_now.tempDATE.apply(lambda x:x[1].split(':')[1])
bicycle_now['LEAS_sec']=bicycle_now.tempDATE.apply(lambda x:x[1].split(':')[2])

In [30]:
bicycle_now.head()

,LEAS_NO,LEAS_STAT,LEAS_DATE,LEAS_STATION,LEAS_DEF_NO,RTN_DATE,RTN_STATION,RTN_DEF_NO,TRNV_QTY,MEMB_DIV,MEMB_NO,TEMP_MEMB_NO,BIKE_TAG,RTN_PROCESS,tempDATE,LEAS_year,LEAS_month,LEAS_day,LEAStime,LEAS_hour,LEAS_min,LEAS_sec
0,15945541,2,2017-01-01 00:00:41,213,18,2017-01-01 00:13:52,260,17,0.0,6,164203,0.0,1A844000000BB7,NaN,"[2017-01-01, 00:00:41]",2017,01,01,[00:00:41],00,00,41
1,15945542,2,2017-01-01 00:01:03,231,4,2017-01-01 00:50:24,231,17,31039.0,1,187551,0.0,1A844000000494,NaN,"[2017-01-01, 00:01:03]",2017,01,01,[00:01:03],00,01,03
2,15945543,2,2017-01-01 00:01:50,119,14,2017-01-01 01:01:50,0,0,NaN,12,168994,0.0,1A844000000533,1.0,"[2017-01-01, 00:01:50]",2017,01,01,[00:01:50],00,01,50
3,15945544,2,2017-01-01 00:02:09,121,17,2017-01-01 00:15:58,133,14,15490.0,12,183971,0.0,1A844000000731,NaN,"[2017-01-01, 00:02:09]",2017,01,01,[00:02:09],00,02,09
4,15945545,2,2017-01-01 00:03:32,320,29,2017-01-01 00:18:44,259,27,0.0,12,167475,0.0,1A84400000F343,NaN,"[2017-01-01, 00:03:32]",2017,01,01,[00:03:32],00,03,32


In [31]:
bicycle_now.tail()

,LEAS_NO,LEAS_STAT,LEAS_DATE,LEAS_STATION,LEAS_DEF_NO,RTN_DATE,RTN_STATION,RTN_DEF_NO,TRNV_QTY,MEMB_DIV,MEMB_NO,TEMP_MEMB_NO,BIKE_TAG,RTN_PROCESS,tempDATE,LEAS_year,LEAS_month,LEAS_day,LEAStime,LEAS_hour,LEAS_min,LEAS_sec
3142683,21718608,2,2019-12-31 23:52:39,222,16,2019-12-31 23:58:56,219,24,0.0,3,236957,0.0,1A84400000F881,NaN,"[2019-12-31, 23:52:39]",2019,12,31,[23:52:39],23,52,39
3142684,21718610,2,2019-12-31 23:53:59,343,14,2019-12-31 23:57:59,311,15,0.0,99,0,4124665.0,1A84400000F9DA,NaN,"[2019-12-31, 23:53:59]",2019,12,31,[23:53:59],23,53,59
3142685,21718611,2,2019-12-31 23:55:27,333,17,2020-01-01 00:06:24,322,5,0.0,99,0,4124667.0,1A84400000048B,NaN,"[2019-12-31, 23:55:27]",2019,12,31,[23:55:27],23,55,27
3142686,21718613,2,2019-12-31 23:59:14,264,4,2020-01-01 00:10:39,210,13,0.0,12,180671,0.0,1A84400000F804,NaN,"[2019-12-31, 23:59:14]",2019,12,31,[23:59:14],23,59,14
3142687,21718614,2,2019-12-31 23:59:30,214,18,2020-01-01 00:10:08,222,3,33169.0,12,215409,0.0,1A84400000F5C7,NaN,"[2019-12-31, 23:59:30]",2019,12,31,[23:59:30],23,59,30


In [35]:
bicycle_now.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142688 entries, 0 to 3142687
Data columns (total 22 columns):
 #   Column        Dtype  
---  ------        -----  
 0   LEAS_NO       int64  
 1   LEAS_STAT     int64  
 2   LEAS_DATE     object 
 3   LEAS_STATION  int64  
 4   LEAS_DEF_NO   int64  
 5   RTN_DATE      object 
 6   RTN_STATION   int64  
 7   RTN_DEF_NO    int64  
 8   TRNV_QTY      float64
 9   MEMB_DIV      object 
 10  MEMB_NO       int64  
 11  TEMP_MEMB_NO  float64
 12  BIKE_TAG      object 
 13  RTN_PROCESS   float64
 14  tempDATE      object 
 15  LEAS_year     object 
 16  LEAS_month    object 
 17  LEAS_day      object 
 18  LEAStime      object 
 19  LEAS_hour     object 
 20  LEAS_min      object 
 21  LEAS_sec      object 
dtypes: float64(3), int64(7), object(12)
memory usage: 527.5+ MB


In [37]:
# tempDATE_2에서 년/월/일 쪼개기
bicycle_now['RTN_year']=bicycle_now.tempDATE_2.apply(lambda x:x[0].split('-')[0])
bicycle_now['RTN_month']=bicycle_now.tempDATE_2.apply(lambda x:x[0].split('-')[1])
bicycle_now['RTN_day']=bicycle_now.tempDATE_2.apply(lambda x:x[0].split('-')[2])

#tempDATE_2에서 시/분/초 쪼개기
bicycle_now['RTNtime']=bicycle_now.tempDATE_2.apply(lambda x:x[1].split())

In [38]:
bicycle_now.head()

,LEAS_NO,LEAS_STAT,LEAS_DATE,LEAS_STATION,LEAS_DEF_NO,RTN_DATE,RTN_STATION,RTN_DEF_NO,TRNV_QTY,MEMB_DIV,MEMB_NO,TEMP_MEMB_NO,BIKE_TAG,RTN_PROCESS,tempDATE,LEAS_year,LEAS_month,LEAS_day,LEAStime,LEAS_hour,LEAS_min,LEAS_sec,tempDATE_2,RTN_year,RTN_month,RTN_day,RTNtime
0,15945541,2,2017-01-01 00:00:41,213,18,2017-01-01 00:13:52,260,17,0.0,6,164203,0.0,1A844000000BB7,NaN,"[2017-01-01, 00:00:41]",2017,01,01,[00:00:41],00,00,41,"[2017-01-01, 00:13:52]",2017,01,01,[00:13:52]
1,15945542,2,2017-01-01 00:01:03,231,4,2017-01-01 00:50:24,231,17,31039.0,1,187551,0.0,1A844000000494,NaN,"[2017-01-01, 00:01:03]",2017,01,01,[00:01:03],00,01,03,"[2017-01-01, 00:50:24]",2017,01,01,[00:50:24]
2,15945543,2,2017-01-01 00:01:50,119,14,2017-01-01 01:01:50,0,0,NaN,12,168994,0.0,1A844000000533,1.0,"[2017-01-01, 00:01:50]",2017,01,01,[00:01:50],00,01,50,"[2017-01-01, 01:01:50]",2017,01,01,[01:01:50]
3,15945544,2,2017-01-01 00:02:09,121,17,2017-01-01 00:15:58,133,14,15490.0,12,183971,0.0,1A844000000731,NaN,"[2017-01-01, 00:02:09]",2017,01,01,[00:02:09],00,02,09,"[2017-01-01, 00:15:58]",2017,01,01,[00:15:58]
4,15945545,2,2017-01-01 00:03:32,320,29,2017-01-01 00:18:44,259,27,0.0,12,167475,0.0,1A84400000F343,NaN,"[2017-01-01, 00:03:32]",2017,01,01,[00:03:32],00,03,32,"[2017-01-01, 00:18:44]",2017,01,01,[00:18:44]


In [44]:
# 대여-반납 시간 구하기
# 데이터유형 변경
#시계열데이터 추출을 위해 'object -> datetime' 으로


bicycle_now["LEAS_DATE"]=pd.to_datetime(bicycle_now['LEAS_DATE'])
bicycle_now["RTN_DATE"]=pd.to_datetime(bicycle_now['RTN_DATE'])

In [49]:
import datetime
from dateutil.relativedelta import relativedelta
from datetime import date


bicycle_now['Diff'] = bicycle_now['RTN_DATE'] - bicycle_now['LEAS_DATE']
bicycle_now['Diff']=bicycle_now['Diff']/np.timedelta64(1,'m')

print(bicycle_now)

          LEAS_NO  LEAS_STAT           LEAS_DATE  ...  RTN_day     RTNtime       Diff
0        15945541          2 2017-01-01 00:00:41  ...       01  [00:13:52]  13.183333
1        15945542          2 2017-01-01 00:01:03  ...       01  [00:50:24]  49.350000
2        15945543          2 2017-01-01 00:01:50  ...       01  [01:01:50]  60.000000
3        15945544          2 2017-01-01 00:02:09  ...       01  [00:15:58]  13.816667
4        15945545          2 2017-01-01 00:03:32  ...       01  [00:18:44]  15.200000
...           ...        ...                 ...  ...      ...         ...        ...
3142683  21718608          2 2019-12-31 23:52:39  ...       31  [23:58:56]   6.283333
3142684  21718610          2 2019-12-31 23:53:59  ...       31  [23:57:59]   4.000000
3142685  21718611          2 2019-12-31 23:55:27  ...       01  [00:06:24]  10.950000
3142686  21718613          2 2019-12-31 23:59:14  ...       01  [00:10:39]  11.416667
3142687  21718614          2 2019-12-31 23:59:30  ... 

- 대여-반납 차를 나타내는 컬럼['Diff']
- 참고 : [Calculate time difference](https://stackoverflow.com/questions/59833174/calculate-time-difference-within-pandas-while-adding-a-new-column)


In [50]:
# 필요없는 컬럼 삭제
del bicycle_now['LEAS_hour']
del bicycle_now['LEAS_min']
del bicycle_now['LEAS_sec']
del bicycle_now['LEAStime']
del bicycle_now['RTNtime']

In [51]:
bicycle_now.head()

,LEAS_NO,LEAS_STAT,LEAS_DATE,LEAS_STATION,LEAS_DEF_NO,RTN_DATE,RTN_STATION,RTN_DEF_NO,TRNV_QTY,MEMB_DIV,MEMB_NO,TEMP_MEMB_NO,BIKE_TAG,RTN_PROCESS,tempDATE,LEAS_year,LEAS_month,LEAS_day,tempDATE_2,RTN_year,RTN_month,RTN_day,Diff
0,15945541,2,2017-01-01 00:00:41,213,18,2017-01-01 00:13:52,260,17,0.0,6,164203,0.0,1A844000000BB7,NaN,"[2017-01-01, 00:00:41]",2017,01,01,"[2017-01-01, 00:13:52]",2017,01,01,13.183333
1,15945542,2,2017-01-01 00:01:03,231,4,2017-01-01 00:50:24,231,17,31039.0,1,187551,0.0,1A844000000494,NaN,"[2017-01-01, 00:01:03]",2017,01,01,"[2017-01-01, 00:50:24]",2017,01,01,49.350000
2,15945543,2,2017-01-01 00:01:50,119,14,2017-01-01 01:01:50,0,0,NaN,12,168994,0.0,1A844000000533,1.0,"[2017-01-01, 00:01:50]",2017,01,01,"[2017-01-01, 01:01:50]",2017,01,01,60.000000
3,15945544,2,2017-01-01 00:02:09,121,17,2017-01-01 00:15:58,133,14,15490.0,12,183971,0.0,1A844000000731,NaN,"[2017-01-01, 00:02:09]",2017,01,01,"[2017-01-01, 00:15:58]",2017,01,01,13.816667
4,15945545,2,2017-01-01 00:03:32,320,29,2017-01-01 00:18:44,259,27,0.0,12,167475,0.0,1A84400000F343,NaN,"[2017-01-01, 00:03:32]",2017,01,01,"[2017-01-01, 00:18:44]",2017,01,01,15.200000


In [55]:
# 필요없는 컬럼 삭제2
#자전거 번호 삭제
del bicycle_now['BIKE_TAG']

KeyError: ignored

In [56]:
#데이터 복구
bicycle = bicycle_now.copy()

In [57]:
bicycle.head(3)

,LEAS_NO,LEAS_STAT,LEAS_DATE,LEAS_STATION,LEAS_DEF_NO,RTN_DATE,RTN_STATION,RTN_DEF_NO,TRNV_QTY,MEMB_DIV,MEMB_NO,TEMP_MEMB_NO,RTN_PROCESS,tempDATE,LEAS_year,LEAS_month,LEAS_day,tempDATE_2,RTN_year,RTN_month,RTN_day,Diff
0,15945541,2,2017-01-01 00:00:41,213,18,2017-01-01 00:13:52,260,17,0.0,6,164203,0.0,NaN,"[2017-01-01, 00:00:41]",2017,01,01,"[2017-01-01, 00:13:52]",2017,01,01,13.183333
1,15945542,2,2017-01-01 00:01:03,231,4,2017-01-01 00:50:24,231,17,31039.0,1,187551,0.0,NaN,"[2017-01-01, 00:01:03]",2017,01,01,"[2017-01-01, 00:50:24]",2017,01,01,49.350000
2,15945543,2,2017-01-01 00:01:50,119,14,2017-01-01 01:01:50,0,0,NaN,12,168994,0.0,1.0,"[2017-01-01, 00:01:50]",2017,01,01,"[2017-01-01, 01:01:50]",2017,01,01,60.000000


In [58]:
# bicycle_now에서 필요없는 컬럼(거치대 번호) 삭제
del bicycle_now['LEAS_DEF_NO']
del bicycle_now['RTN_DEF_NO']

bicycle_now.head(3)

,LEAS_NO,LEAS_STAT,LEAS_DATE,LEAS_STATION,RTN_DATE,RTN_STATION,TRNV_QTY,MEMB_DIV,MEMB_NO,TEMP_MEMB_NO,RTN_PROCESS,tempDATE,LEAS_year,LEAS_month,LEAS_day,tempDATE_2,RTN_year,RTN_month,RTN_day,Diff
0,15945541,2,2017-01-01 00:00:41,213,2017-01-01 00:13:52,260,0.0,6,164203,0.0,NaN,"[2017-01-01, 00:00:41]",2017,01,01,"[2017-01-01, 00:13:52]",2017,01,01,13.183333
1,15945542,2,2017-01-01 00:01:03,231,2017-01-01 00:50:24,231,31039.0,1,187551,0.0,NaN,"[2017-01-01, 00:01:03]",2017,01,01,"[2017-01-01, 00:50:24]",2017,01,01,49.350000
2,15945543,2,2017-01-01 00:01:50,119,2017-01-01 01:01:50,0,NaN,12,168994,0.0,1.0,"[2017-01-01, 00:01:50]",2017,01,01,"[2017-01-01, 01:01:50]",2017,01,01,60.000000


In [59]:
# 회원/ 비회원 번호는 무의미. 단 구분은 필요
del bicycle_now['MEMB_NO']
del bicycle_now['TEMP_MEMB_NO']

bicycle_now.head(3)


,LEAS_NO,LEAS_STAT,LEAS_DATE,LEAS_STATION,RTN_DATE,RTN_STATION,TRNV_QTY,MEMB_DIV,RTN_PROCESS,tempDATE,LEAS_year,LEAS_month,LEAS_day,tempDATE_2,RTN_year,RTN_month,RTN_day,Diff
0,15945541,2,2017-01-01 00:00:41,213,2017-01-01 00:13:52,260,0.0,6,NaN,"[2017-01-01, 00:00:41]",2017,01,01,"[2017-01-01, 00:13:52]",2017,01,01,13.183333
1,15945542,2,2017-01-01 00:01:03,231,2017-01-01 00:50:24,231,31039.0,1,NaN,"[2017-01-01, 00:01:03]",2017,01,01,"[2017-01-01, 00:50:24]",2017,01,01,49.350000
2,15945543,2,2017-01-01 00:01:50,119,2017-01-01 01:01:50,0,NaN,12,1.0,"[2017-01-01, 00:01:50]",2017,01,01,"[2017-01-01, 01:01:50]",2017,01,01,60.000000


In [60]:
bicycle_now.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142688 entries, 0 to 3142687
Data columns (total 18 columns):
 #   Column        Dtype         
---  ------        -----         
 0   LEAS_NO       int64         
 1   LEAS_STAT     int64         
 2   LEAS_DATE     datetime64[ns]
 3   LEAS_STATION  int64         
 4   RTN_DATE      datetime64[ns]
 5   RTN_STATION   int64         
 6   TRNV_QTY      float64       
 7   MEMB_DIV      object        
 8   RTN_PROCESS   float64       
 9   tempDATE      object        
 10  LEAS_year     object        
 11  LEAS_month    object        
 12  LEAS_day      object        
 13  tempDATE_2    object        
 14  RTN_year      object        
 15  RTN_month     object        
 16  RTN_day       object        
 17  Diff          float64       
dtypes: datetime64[ns](2), float64(3), int64(4), object(9)
memory usage: 431.6+ MB


- 연도별로 구분되어 있는 원래 데이터
- 자전거 스테이션을 기준으로 그룹 바이..?

In [64]:
bicycle_now['MEM']=0

for m in range(0,len(bicycle_now)):
  if bicycle_now.iloc[m,7]=="99" : 
    bicycle_now.iloc[m,19] = 0
  else:
    bicycle_now.iloc[m,19] = 1

IndexError: ignored

In [ ]:
# 연도별 카운트
fig = plt.figure(figsize=[12,10])
ax1= fig.add_subplot(2,2,1)
ax1= sns.barplot(x='LEAS_year', y='LEAS_station', data=bicycle.groupby('LEAS_year')['LEAS_station'].)